#**MODELO RECOMENDADOR COMPARTIR TRAYECTO**
El presente modelo se ha diseñado para realizar una recomendación para compartir trayecto entre ciudadanos cercanos. 

Esta basado en un recomendador item-to-item para las rutas de los diferentes ciudadanos.

Los datos de ciudadanos y rutas han sido simulados.

#0.Inicialización

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importamos las librerías pandas y numpy que nos servirán para el manejo de tablas 
# tipo data-frames y operaciones matemáticas

# Importamos el TF-IDF de la librería sklearn
# Importamos la función para calcular la distancia coseno (la que vimos en clase)

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import nltk
from nltk.corpus import stopwords

#1.Datos

In [ ]:
#%% Carga del dataframe.
df_recomendador =pd.read_excel('/content/drive/MyDrive/PSIA-proyecto-Madrid/recomruta/recomendador_rutas.xlsx', index_col=0)  
# La función head de pandas nos sirve para imprimir por pantalla las primeras filas del dataframe
df_recomendador.head()

,usuario,nombre,apellido,latitud,longitud,Origen,Destino,ruta
id,,,,,,,,
1,1001,Isidro,Labrador,40.467114,-3.689158,Plaza de castilla,Plaza de España,Plaza de Castilla 28046 Madrid Dirígete hacia ...
2,1035,Pepita,Perez,40.471917,-3.682183,Chamartin,Plaza de Marques de Salamanca,Estación de Madrid-Chamartín 28036 Madrid Sigu...
3,1122,Paloma,La Patrona,40.470901,-3.690286,calle cedros 92,calle goya 92,"Calle Cedros, 92 28029 Madrid Toma Calle de lo..."
4,1199,Manolo,El del Bombo,40.459439,-3.690241,Plaza de cuzco,plaza de cibeles,Plaza de Cuzco 28046 Madrid Dirígete hacia el ...
5,1245,Elena,Nieto del Bosque,40.467206,-3.684241,Calle Mateo Inurria 19,NaN,"Calle Mateo Inurria, 19 28036 Madrid Dirígete..."


In [ ]:
# ¿Qué información tenemos disponible? El atributo "columns" de un dataframe lista sus columnas
df_recomendador.columns

Index(['usuario', 'nombre', 'apellido', 'latitud', 'longitud', 'Origen',
       'Destino', 'ruta'],
      dtype='object')

In [ ]:
# Consultamos id Isidro Labrador

df_recomendador[df_recomendador['nombre'] == 'Isidro']

,usuario,nombre,apellido,latitud,longitud,Origen,Destino,ruta
id,,,,,,,,
1,1001,Isidro,Labrador,40.467114,-3.689158,Plaza de castilla,Plaza de España,Plaza de Castilla 28046 Madrid Dirígete hacia ...


In [ ]:
# Consultamos el Trayecto usuario 1:

df_recomendador.loc[1, 'ruta']

'Plaza de Castilla 28046 Madrid Dirígete hacia el sureste en Av. de Asturias hacia Plaza Castilla 9 s (32 m) Toma Paseo de la Castellana. 58 s (400 m) Sigue por Paseo de la Castellana. 5 min (2,4 km) Toma Calle de Ríos Rosas hacia Av. de Filipinas/Túnel de Ríos Rosas. 2 min (1,0 km) Continúa por Av. de Filipinas/Túnel de Ríos Rosas  Continúa hacia Av. de Filipinas\n1 min (600 m) Toma Calle de Guzmán el Bueno hacia Calle de Alberto Aguilera. 4 min (1,2 km) Conduce hasta C. de la Princesa. 4 min (850 m) Plaza de España 28008 Madrid'

#2.Matriz recomendador

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.data import load
from nltk.stem import SnowballStemmer
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
spanish_stopwords = stopwords.words('spanish')
stemmer = SnowballStemmer('spanish')
non_words = list(punctuation)
non_words.extend(['¿', '¡'])
non_words.extend(['0','1','2','3','4','5','6','7','8','9'])
non_words.extend(map(str,range(10)))
stemmer = SnowballStemmer('spanish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Definimos nuestro TF-IDF con los siguientes parámetros:

# El rango de n-gramas marca cuántas palabras consecutivas consideraremos. Se pueden utilizar varias a la vez.
# Si ponemos un rango (1, 2) significa monogramas y bigramas. Si ponemos (3,3) significa sólo trigramas.

# min_df y max_df marca la limpieza de palabras demasiado habituales o demasiado raras para aportar información

# stop_words selecciona la lista de stop words a usar, en este caso la del inglés

# token_pattern define lo que se considera una "palabra". En este caso cualquier combinación de letras

#stopword list to use
df_recomendador['ruta']=df_recomendador['ruta'].str.replace('\d+', '') 

tf = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     #min_df=0, #0.00001,
                     #max_df=1, #0.9999,
                     stop_words=spanish_stopwords, 
                     token_pattern= r'([a-zA-Z]{1,})')

df_recomendador['ruta'] = df_recomendador['ruta'].fillna('')

tfidf_matrix = tf.fit_transform(df_recomendador['ruta'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ais', 'amos', 'an', 'as', 'bamos', 'est', 'estuvi', 'fu', 'hab', 'habr', 'hubi', 'is', 'l', 'm', 'n', 'qu', 'ramos', 's', 'semos', 'ser', 't', 'tambi', 'ten', 'tendr', 'teng', 'tuvi'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
# ¿Qué tamaño tiene nuestra matriz? ¿A qué se corresponden esos números?

tfidf_matrix.shape

(5, 269)

In [ ]:
# Un ejemplo de las "palabras" obtenidas:

tf.get_feature_names()[90:110]

['dir gete',
 'domingos',
 'domingos km',
 'eloy',
 'eloy gonzalo',
 'emilio',
 'emilio castelar',
 'espa',
 'espa madrid',
 'est',
 'est derecha',
 'est izquierda',
 'est n',
 'estaci',
 'estaci n',
 'filipinas',
 'filipinas min',
 'filipinas t',
 'general',
 'general mart']

In [ ]:
# La función "lineal_kernel" calcula la distancia coseno como vimos en clase
# Al calcularla de toda la matriz contra si misma, estamos calculando de una vez todas las distancias posibles

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[0]

array([1.        , 0.11361498, 0.35906968, 0.23744344, 0.25846855])

In [ ]:
# Para elegir ahora las ruta mas parecida de un usuario con otro en su zona:

smd = df_recomendador.reset_index()
usuarios = df_recomendador['usuario']
print(usuarios)
indices = pd.Series(smd.index, index=smd['usuario'])

def get_recommendations(user):
    
    idx = indices[user] # idx es el índice del usuario que busca recomendador
    
    # Calculamos toda la lista de similaridades de todas las películas contra la que buscamos
    sim_scores = list(enumerate(cosine_sim[idx])) 
    
    # Ordenamos la lista de similaridades (sorted) y nos quedamos con los índices de esas películas
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Seleccionamos las primeras 10 de la lista
    sim_scores = sim_scores[0:10] # ¿Empezamos en 0 o empezamos en 1?
    
    # La función devuelve los nombres de las películas, no sus valores, así que seleccionamos sus índices
    friend_indices = [i[0] for i in sim_scores]
    
    # Devolvemos los usuarios
    return usuarios.iloc[friend_indices]

id
1    1001
2    1035
3    1122
4    1199
5    1245
Name: usuario, dtype: int64


#3.Resultados

In [ ]:
print(get_recommendations(1001).head())


id
1    1001
3    1122
5    1245
4    1199
2    1035
Name: usuario, dtype: int64


In [ ]:
print('El usuario',df_recomendador.loc[1, 'nombre'],df_recomendador.loc[1, 'apellido'],'podría compartir transporte con', 
      df_recomendador.loc[3, 'nombre'],df_recomendador.loc[3, 'apellido'],'.')

El usuario Isidro Labrador podría compartir transporte con Paloma La Patrona .
